In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs found. Training will default to CPU.")

No GPUs found. Training will default to CPU.


In [2]:
data_dir = 'data'

In [3]:
# Define parameters
img_width, img_height = 128, 128
batch_size = 32

In [4]:
# Create an ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1765  # Approx 15% for validation, when combined with 70% training
)


In [5]:
# Create an ImageDataGenerator for testing (no augmentation, just scaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
# Prepare iterators
train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Set as training data
)


Found 9783 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'  # Set as validation data
)


Found 2096 images belonging to 2 classes.


In [8]:
test_generator = test_datagen.flow_from_directory(
    data_dir + '/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.


In [12]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 28800)             0         
                                                                 
 dense_1 (Dense)             (None, 64)                1

In [13]:
import tensorflow as tf
import keras
import scipy

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)
print("Scipy Version:", scipy.__version__)

epochs = 10
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)


TensorFlow Version: 2.13.0
Keras Version: 2.13.1
Scipy Version: 1.10.1
Epoch 1/10
305/305 [==============================] - 73s 237ms/step - loss: 0.5152 - accuracy: 0.7520 - val_loss: 0.5300 - val_accuracy: 0.7409
Epoch 2/10
305/305 [==============================] - 73s 238ms/step - loss: 0.4051 - accuracy: 0.8175 - val_loss: 0.5045 - val_accuracy: 0.7769
Epoch 3/10
305/305 [==============================] - 76s 248ms/step - loss: 0.3973 - accuracy: 0.8281 - val_loss: 0.5336 - val_accuracy: 0.7543
Epoch 4/10
305/305 [==============================] - 76s 250ms/step - loss: 0.3820 - accuracy: 0.8384 - val_loss: 0.4868 - val_accuracy: 0.7841
Epoch 5/10
305/305 [==============================] - 76s 248ms/step - loss: 0.3729 - accuracy: 0.8385 - val_loss: 0.5059 - val_accuracy: 0.7486
Epoch 6/10
305/305 [==============================] - 75s 245ms/step - loss: 0.3633 - accuracy: 0.8424 - val_loss: 0.4804 - val_accuracy: 0.7716
Epoch 7/10
305/305 [==============================] - 32s 1

In [14]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')


63/63 [==============================] - 2s 23ms/step - loss: 0.3164 - accuracy: 0.8785
Test accuracy: 0.8784999847412109
